In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from datetime import datetime

from selenium.webdriver.common.keys import Keys
import time



In [2]:
driver = webdriver.Edge(r"C:\Users\rrd\Desktop\doubt\msedgedriver.exe")

1. Write a python program which searches all the product under a particular product from www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If 
user input is ‘guitar’. Then search for guitars.
2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has 
less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.






In [3]:
url="https://www.amazon.in"
driver.get(url)


In [4]:
product_vertical=input("Enter product vertical that needs to be searched:")
driver.find_element_by_id("twotabsearchtextbox").send_keys(product_vertical)
driver.find_element_by_id("nav-search-submit-button").click()


Enter product vertical that needs to be searched:laptop


In [5]:
#create a function to get the details of all products of first three pages
urls=[]
for a in range(3):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        urls.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break
        
        
# create empty list to store data
brand_name=[]
name_of_the_product=[]
rating=[]
no_of_ratings=[]
price=[]
expected_delivery=[]
return_exchange=[]
other_details=[]
availability=[]
product_url=[]

# traverse all URL one by one
for i in urls:
    driver.get(i)
    product_url.append(i)
    driver.implicitly_wait(5)
    
    
#get the brand name
    try: brand_name.append(driver.find_element_by_id("productTitle").text.split(' ')[0]) 
    except: brand_name.append('-')
    driver.implicitly_wait(5)
    
#get the name of product
    try: 
        string=''
        for i in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+i
        name_of_the_product.append(string)
    except: name_of_the_product.append('-')
    driver.implicitly_wait(5)
    
#the rating of the product
    try: rating.append(driver.find_element_by_id("acrPopover").get_attribute('title')) 
    except: rating.append('-')
    driver.implicitly_wait(5)
    
#the number of ratings
    try: no_of_ratings.append(driver.find_element_by_id("acrCustomerReviewText").text) 
    except: no_of_ratings.append('-')
    driver.implicitly_wait(5)
    
# the price of the product
    try: 
        try: price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try: price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except: price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except: price.append('-')
    driver.implicitly_wait(5)
    
# the details of return/exchnage of the product
    try: 
        return_row=driver.find_element_by_xpath("//div[@class='a-row icon-farm-wrapper']").find_elements_by_xpath(".//div")
        found=False
        for i in return_row:
            if(i.get_attribute('data-name')=='RETURNS_POLICY'):
                found=True
                return_exchange.append(i.find_element_by_xpath(".//span[1]/div[2]/a").text)
        if(found==False):
            return_exchange.append('-')
    except: return_exchange.append('-')
    driver.implicitly_wait(5)
    
# the expected delivery date
    try: expected_delivery.append(driver.find_element_by_id("ddmDeliveryMessage").find_element_by_xpath(".//b").text)
    except: expected_delivery.append('-')
    driver.implicitly_wait(5)
    
# the availablity of the product
    try: 
        try: availability.append(driver.find_element_by_id("availability").find_element_by_xpath(".//span").text)
        except: availability.append(driver.find_element_by_id("deal_availability").find_element_by_xpath(".//div/span").text)
    except: availability.append('-')
    driver.implicitly_wait(5)
    
# other details
    try:
        details=[i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")] 
        other_details.append(details[0])
    except: other_details.append('-')
    driver.implicitly_wait(5)

#  create a DataFrame for our data
amazon=pd.DataFrame({'Brand Name':brand_name,'Name of Product':name_of_the_product,'Rating':rating,'No. of Ratings':no_of_ratings,'Price':price,
             'Return/Exchange':return_exchange,'Expected Delivery':expected_delivery,'Availability':availability,'Other details':other_details,
             'Product URL':product_url})

# save the dataframe in csv file
amazon.to_csv('products.csv')




In [6]:
amazon


,Brand Name,Name of Product,Rating,No. of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other details,Product URL
0,AVITA,Essential NE14A2INC433-MB 14-inch Laptop (Cel...,3.9 out of 5 stars,467 ratings,"₹ 17,990.00",7 Days Replacement,-,In stock.,Brand AVITA---Manufacturer Alco Electronics Li...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Lenovo,Legion 5 AMD Ryzen 5 4600H 15.6 inch Full HD ...,4.8 out of 5 stars,51 ratings,"₹ 67,990.00",7 Days Replacement,-,Available to ship in 1-2 days.,"Brand Lenovo---Manufacturer Lenovo, One of the...",https://www.amazon.in/gp/slredirect/picassoRed...
2,HP,15 Entry Level 15.6-inch HD Laptop (AMD 3020e...,3.9 out of 5 stars,442 ratings,"₹ 23,990.00",7 Days Replacement,-,"In stock on April 25, 2021.","Brand HP---Manufacturer hp, HP---Model Name 15...",https://www.amazon.in/HP-15-6-inch-Laptop-Wind...
3,Dell,"Inspiron 3502 15.6"" HD Display Laptop (Pentiu...",4.7 out of 5 stars,4 ratings,"₹ 27,390.00",7 Days Replacement,-,In stock.,"Brand Dell---Manufacturer Dell India Pvt Ltd, ...",https://www.amazon.in/Dell-Inspiron-Integrated...
4,Lenovo,IdeaPad Slim 3 Intel Celeron N4020 15.6-inch ...,4.0 out of 5 stars,112 ratings,"₹ 25,490.00",7 Days Replacement,-,Only 1 left in stock.,"Brand Lenovo---Manufacturer Lenovo, One of the...",https://www.amazon.in/Lenovo-IdeaPad-15-6-inch...
...,...,...,...,...,...,...,...,...,...,...
61,Lenovo,"IdeaPad S145 Intel Core i5 10th Gen 15.6"" Ful...",3.0 out of 5 stars,1 rating,"₹ 48,990.00",7 Days Replacement,-,In stock.,"Brand Lenovo---Manufacturer Lenovo, One of the...",https://www.amazon.in/Lenovo-IdeaPad-Windows10...
62,HP,Envy x360 Convertible Touchscreen 13.3-inch F...,4.6 out of 5 stars,119 ratings,"₹ 71,990.00",7 Days Replacement,-,In stock.,"Brand HP---Manufacturer hp, Manufactured by on...",https://www.amazon.in/HP-Convertible-Touchscre...
63,HP,15 db1069AU 15.6-inch Laptop (3rd Gen Ryzen 3...,3.7 out of 5 stars,820 ratings,-,7 Days Replacement,-,Available from these sellers.,"Brand HP---Manufacturer hp, Manufactured by on...",https://www.amazon.in/HP-db1069AU-15-6-inch-Wi...
64,HP,15 11th Gen Intel Core i3 Processor 15.6-inch...,4.0 out of 5 stars,103 ratings,"₹ 46,990.00",7 Days Replacement,-,In stock.,"Brand HP---Manufacturer hp, HP India Sales Pvt...",https://www.amazon.in/gp/slredirect/picassoRed...


3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.



In [7]:
#load the URL
driver=webdriver.Edge(r"C:\Users\rrd\Desktop\doubt\msedgedriver.exe")
driver.maximize_window()
url=('https://images.google.com/')
driver.get(url)
driver.implicitly_wait(2)

In [9]:

def get_urls(keywords):
        driver.get(url)
        
        #Let's enter the details in the search column and click search button
        search_bar=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
        search_bar.send_keys(keywords)
        button=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
        button.click()
        driver.implicitly_wait(5)
        
        
#few pages to scrape images from them
        for G in range(12):
            driver.find_element_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']").send_keys(Keys.PAGE_DOWN)
        time.sleep(5)
        picture=[imgage for imgage in driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']")[:100]]
        image_urls=[]
        print(len(picture))
        if(len(picture)==100):
            for i in picture:
                try:
                    i.click()
                    image_urls.append(i.get_attribute('href'))
                except:
                    image_urls.append('Not Available')
            return image_urls
        else:
            print('Image Not Available')


In [10]:
#scrape the url's of all images
fruits=get_urls('Fruits')
cars=get_urls('Cars')
machine_learning=get_urls('Machine Learning')


100
100
100


In [11]:
#create a DataFrame for our data
Image_URL=pd.DataFrame({})
Image_URL['Fruits']=fruits
Image_URL['Cars']=cars
Image_URL['Machine Learning']=machine_learning
Image_URL

,Fruits,Cars,Machine Learning
0,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
1,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
2,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
3,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
4,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
...,...,...,...
95,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
96,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
97,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
98,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...


4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. 
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe 
and CSV.





In [12]:
# load the drivers and URL
driver=webdriver.Edge(r"C:\Users\rrd\Desktop\doubt\msedgedriver.exe")
driver.maximize_window()
driver.get('https://www.flipkart.com/')
driver.implicitly_wait(3)

#  enter the details in the search column
product=input('Enter the product vertival that needs to be searched : ')
driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
driver.implicitly_wait(3)
driver.find_element_by_xpath('//input[@title="Search for products, brands and more"]').send_keys(str(product))
driver.find_element_by_xpath('//button[@type="submit"]').click()
driver.implicitly_wait(3)


Enter the product vertival that needs to be searched : smartphone


In [13]:
# create an empty list to store the scrape data
product_urls=[]
brand_name=[]
smartphone_name=[]
smartphone_color=[]
RAM=[]
ROM=[]
primary_camera=[]
secondary_camera=[]
display_size=[]
display_resolution=[]
processor=[]
processor_core=[]
battery_capacity=[]
price=[]

# define a function
links=driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')

# scrape product URL
for i in links:
    product_urls.append(i.get_attribute('href'))
for i in product_urls:
    driver.get(i)
    driver.implicitly_wait(4)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
    try:
        
# scrape brand name
        title=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split()
        brand_name.append(title[0])
    except:
        brand_name.append('-')
    try:
        
# scrape smartohone name
        smartphone_name.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[3]/td[2]').text)
    except:
        smartphone_name.append('-')
    try:
        
# scrape smartphone colour
        smartphone_color.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[4]/td[2]').text)
    except:
        smartphone_color.append('-')
    try:
        
# scrape smartphone RAM
        RAM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]').text)
    except:
        RAM.append('-')
    try:
        
# scrape smartphone ROM
        ROM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]').text)
    except:
        ROM.append('-')
    try:
        
# scrape smartphone primary camera details
        primary_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]').text)
    except:
        primary_camera.append('-')
    try:
        
# scrape smartphone secondary camera details
        secondary_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]').text)
    except:
        secondary_camera.append('-')
    try:
        
# scrape smartphone display size
        display_size.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]').text)
    except:
        display_size.append('-')
    try:
        
# scrape smartphone screen resolution
        display_resolution.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]').text)
    except:
        display_resolution.append('-')
    try:
        
# scrape smartphone processor
        processor.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]').text)
    except:
        processor.append('-')
    try:
        
# scrape smartphone processor core
        processor_core.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[3]/td[2]').text)
    except:
        processor_core.append('-')
    try:
        
# scrape smartphone battery capacity
        battery_capacity.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr[1]/td[2]').text)
    except:
        battery_capacity.append('-')
    try:
        
# scrape smartphone price
        price.append(driver.find_element_by_xpath('//div[@class="dyC4hf"]/div/div/div').text)
    except:
        price.append('-')


In [14]:
#  create a DataFrame for our data
flipkart=pd.DataFrame({})
flipkart['Brand Name']=brand_name
flipkart['Smartphone Name']=smartphone_name
flipkart['Smartphone Colour']=smartphone_color
flipkart['RAM']=RAM
flipkart['Storage (ROM)']=ROM
flipkart['Primray Camera']=primary_camera
flipkart['Secondary Camera']=secondary_camera
flipkart['Display Size']=display_size
flipkart['Display Resolution']=display_resolution
flipkart['Processor']=processor
flipkart['Processor Cores']=processor_core
flipkart['Battery Capacity']=battery_capacity
flipkart['Price']=price
flipkart['Product URL']=product_urls
flipkart


,Brand Name,Smartphone Name,Smartphone Colour,RAM,Storage (ROM),Primray Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,Product URL
0,REDMI,9i,Midnight Black,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,Brand Warranty of 1 Year Available for Mobile ...,"₹7,999",https://www.flipkart.com/redmi-9i-midnight-bla...
1,REDMI,9i,Nature Green,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,Brand Warranty of 1 Year Available for Mobile ...,"₹7,999",https://www.flipkart.com/redmi-9i-nature-green...
2,OPPO,A12,Blue,4 GB,64 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹8,990",https://www.flipkart.com/oppo-a12-blue-64-gb/p...
3,REDMI,9i,Sea Blue,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,Brand Warranty of 1 Year Available for Mobile ...,"₹7,999",https://www.flipkart.com/redmi-9i-sea-blue-64-...
4,OPPO,A12,Blue,3 GB,32 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹7,990",https://www.flipkart.com/oppo-a12-blue-32-gb/p...
5,OPPO,A12,Deep Blue,3 GB,32 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹7,990",https://www.flipkart.com/oppo-a12-deep-blue-32...
6,OPPO,A12,Deep Blue,4 GB,64 GB,13MP + 2MP,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,4230 mAh,"₹8,990",https://www.flipkart.com/oppo-a12-deep-blue-64...
7,realme,C20,Cool Grey,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,76.4 mm,"₹6,799",https://www.flipkart.com/realme-c20-cool-grey-...
8,realme,C20,Cool Blue,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,76.4 mm,"₹6,799",https://www.flipkart.com/realme-c20-cool-blue-...
9,POCO,M3,Cool Blue,6 GB,64 GB,48MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,77.3 mm,"₹10,999",https://www.flipkart.com/poco-m3-cool-blue-64-...


In [15]:
#Let's save the dataframe in csv file
flipkart.to_csv('flipkart_smartphone.csv')


5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.



In [16]:
#creation and opening the url 
driver = webdriver.Edge(r"C:\Users\rrd\Desktop\doubt\msedgedriver.exe")
url= "http://maps.google.com"
driver.get(url)
time.sleep(2)

In [17]:
#Entering the search box 
city_tag=driver.find_element_by_id('searchboxinput')
city_tag.clear()
# entering the city name
city_name="Nellore"
city_tag.send_keys(city_name,Keys.ENTER)

time.sleep(3)

spliturl=[]
# creating a variable str which holds the url of the city searched 
str = driver.current_url
time.sleep(3)
print(str)
# Splitting on encountering @ value
spliturl=str.split("@")
# Again splitting at the delimiter "," and maxsplit is any number of time no limit
latlong=spliturl[1].split(",",maxsplit=-1)
# Holding the first array of the latlong as latitude and the second value as longiturde
latitude=latlong[0]
longitude=latlong[1]
print("Place:",city_name, " Latitude value is :",latitude,"and longitude value is : +",longitude)


https://www.google.com/maps/place/Nellore,+Andhra+Pradesh/@14.4291158,79.9106638,12z/data=!3m1!4b1!4m5!3m4!1s0x3a4c8cca0e958771:0xd3036c2025161f55!8m2!3d14.4425987!4d79.986456
Place: Nellore  Latitude value is : 14.4291158 and longitude value is : + 79.9106638


6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 –September 20) from trak.in.



In [18]:
driver = webdriver.Edge(r"C:\Users\rrd\Desktop\doubt\msedgedriver.exe")
driver.get('https://trak.in/')
driver.implicitly_wait(5)


In [19]:
#enter the details in the search column
search_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(search_button)


In [20]:
Date=[]
Startup=[]
Industry=[]
SubVertical=[]
Location=[]
Investor=[]
Investment=[]
Amount=[]


In [22]:

for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()
    
# scrape the date
    date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date:
        Date.append(d.text)
        
# scrape the detaisl
    startup = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in startup:
        Startup.append(n.text)
    
# scrape the industry details
    industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in industry:
        Industry.append(n.text)
    
# scrape the subvertical details
    subvertical = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in subvertical:
        SubVertical.append(s.text)

# scrape the location details
    location = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in location:
        Location.append(l.text)
    
# scrape the investors detils
    investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in investor:
        Investor.append(n.text)
    
#scrape the investment details
    investment = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in investment:
        Investment.append(n.text)
    
#scrape the amount
    amtount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amtount:
        Amount.append(a.text)

In [23]:
#create a DataFrame for our data
trak_fund=pd.DataFrame({})
trak_fund['Date']=Date
trak_fund['Startup']=Startup 
trak_fund['Industry']=Industry
trak_fund['SubVertical']=SubVertical
trak_fund['Location']=Location
trak_fund['Investor']=Investor
trak_fund['Investment']=Investment
trak_fund['Amount']=Amount


In [24]:
trak_fund


,Date,Startup,Industry,SubVertical,Location,Investor,Investment,Amount
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in. 

In [25]:
driver=webdriver.Edge(r"C:\Users\rrd\Desktop\doubt\msedgedriver.exe")

driver.get("https://www.digit.in/top-products/best-gaming-laptops-40.html")


In [26]:
#  the name of the laptop from the values of the selector "h3"
main_tag=driver.find_elements_by_css_selector("h3")
name=[]
for laptop in main_tag:
    name.append(laptop.text)

#  all the elements from the class name "Spcs-details"
specs_tag=driver.find_elements_by_class_name("Spcs-details")

#  the description is in the preceding sibling under the element "p"
descrpt=[]
for specs in specs_tag:
    descrpt.append(specs.find_element_by_xpath("./preceding-sibling::p").text)

time.sleep(2)
# the list to hold the corresponding attributes 
OS=[]
Display=[]
Processor=[]
Memory=[]
HD=[]
RAM=[]
HDD=[]
Weight=[]
Dimension=[]
Graphics=[]
Price=[]
Description=[]
count=0
# under the specs_tag all the attributes are stored. Corresponding values are stored by traversing under each using the for loop
for specs in specs_tag:
    OS.append(specs.find_element_by_xpath("./table/tbody/tr[3]/td[3]").text)
    Display.append(specs.find_element_by_xpath("./table/tbody/tr[4]/td[3]").text)
    Processor.append(specs.find_element_by_xpath("./table/tbody/tr[5]/td[3]").text)
    Memory.append(specs.find_element_by_xpath("./table/tbody/tr[6]/td[3]").text)
    Weight.append(specs.find_element_by_xpath("./table/tbody/tr[7]/td[3]").text)
    Dimension.append(specs.find_element_by_xpath("./table/tbody/tr[8]/td[3]").text)
    Graphics.append(specs.find_element_by_xpath("./table/tbody/tr[9]/td[3]").text)
    Description.append(specs.find_element_by_xpath("./preceding-sibling::p").text)
    # HD values are stored after splitting at the delimiter "/" 
    for i in Memory:
        HD.append(i.split("/"))
# for all the range till the number of values of Memory list, we are appending first value into HDD and the Second value into RAM attribute
for j in range(0,len(Memory)):
    HDD.append(HD[j][0])
    RAM.append(HD[j][1])
# Getting the pricetag element fro mthe class name smprice and appending it to the Price list
price_tag=driver.find_elements_by_class_name("smprice")
for i in price_tag:
    Price.append(i.text)

#Creating the dataframe and storing the list into the dataframes
Gaming_laptop=pd.DataFrame({})
Gaming_laptop["Name of the Laptop"]=name
Gaming_laptop["OS"]=OS
Gaming_laptop["Display type and Size"]=Display
Gaming_laptop["Processor"]=Processor
Gaming_laptop["RAM"]=RAM
Gaming_laptop["Hard Disk size"]=HDD
Gaming_laptop["Weight"]=Weight
Gaming_laptop["Graphic card"]=Graphics
Gaming_laptop["Description"]=Description
Gaming_laptop

,Name of the Laptop,OS,Display type and Size,Processor,RAM,Hard Disk size,Weight,Graphic card,Description
0,MSI GT76 TITAN DT 9SG,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,64 GBGB DDR4,1 TB HDD,4.2 kg,NVIDIA GeForce RTX 2080,Sporting a desktop graphics Intel Core i9-9900...
1,ALIENWARE 17 AREA-51M,Windows 10 Pro,"17.3"" (FHD (1920 x 1080))",Intel 9th Gen Core i9-9900K | 5000 MHz,64 GBGB DDR4,1 TB HDD,3.87 Kg,NVIDIA GeForce RTX 2080,The Alienware Area-51m was certainly the first...
2,HP OMEN 15 2020,Windows 10 Home,"15.6"" (1920 x 1080)",Intel i7-10750H 10th Gen | 1.6GHz,32GB DDR4,1 TB PCIe SSD,5.40,Nvidia GeForce GTX 1650Ti,The new HP Omen 15 comes in both 10th generati...
3,ASUS ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,64 GBGB DDR4,1 TB HDD,1.65,NVIDIA GeForce RTX 2060,The Asus Zephyrus G14 is a first-of-its-kind g...
4,LENOVO LEGION Y540,Windows 10 Home,"15.6"" (1920 X 1080)",9th Generation Core Intel I7-9750H | 2.6 Ghz,32GB DDR4,1 TB PCIe SSD,2.3,NVIDIA® GeForce RTX™ 2060,The Lenovo Legion Y540 is powered by an Intel ...
5,ASUS ROG ZEPHYRUS G GA502,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen 7 Quad Core 3750H | 2.3 GHz,16 GBGB DDR4,512 GB SSD,2.2,NVIDIA Geforce GTX 1660 Ti,The Ryzen 7 3750H powered Asus ROG Zephyrus G ...
6,ASUS ROG ZEPHYRUS S GX531,Windows 10 Home,"15.6"" (1920 X 1080)",9th Gen Intel Core i7-8750H | 2.2 GHz,64 GBGB DDR4,1 TB HDD,2.1,NVIDIA® GeForce RTX™ 2070 (Max-Q),The Asus Zephyrus S (GX531) manages to get a f...
7,MSI GT83VR 7RE TITAN SLI,Windows 10 Home 64 bit,"18.4"" (1920 x 1080)",Intel CM238 Core i7-7820HK+CM238 7th Gen | 3.5GHz,32GB DDR4,1 TB PCIe SSD,5.5,Dual GTX1070,MSI does not have a dual GTX 1080 gaming lapto...
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,16 GBGB DDR4,512 GB SSD,2.4,NVIDIA GeForce RTX 2070 Max-Q,The machine is powered by an Intel Core i7-108...
9,DELL G3,Windows 10 Home Plus,15.6 MP | NA,8th Gen Intel core i5-8300H | 2.3GHz,16 GBGB DDR4,1 TB SSD,2.53,NVidia GeForce GTX 1050,The Dell G3 is a capable entry-level gaming la...
